In [1]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import random
import sys
import io
import re

pd.set_option('display.max_row', 50)
pd.set_option('display.max_columns', 100)

In [2]:
data = pd.read_csv('CVdata_r7.csv')
data

,id,content
0,1,봉사활동을 참여하게 된 후로부터 저는 저의 도움으로 다른 사람들이 행복해지는 모습을...
1,2,노력파 현재진행형 인재저는 컴퓨터 전기전자과를 전공하여 전기 전자와 관련된 여러 과...
2,3,설계의 꿈을 갖고 기초공사를 마치다제가 설계 직무에 적합한 이유로 3가지가 있습니다...
3,4,감동을 주는 엔지니어응용공학설계 프로젝트에서 차량 프로젝트로 설계와 구조해석을 하면...
4,5,기업 솔루션 종결자 후지제록스에서 꿈을 펼치다!후지제록스는 많은 기업이 직면해 있는...
...,...,...
1265,1266,수동적이고 기계적인 태도로 주어진 기본 업무만 처리하는 사람은 능동적으로 업무를 완...
1266,1267,우리 사회가 고령화 사회로 진입하게 되면서 사람들은 단순히 오래 사는 것보다는 건강...
1267,1268,귀사의 사회공헌활동에는 역동성이 있습니다. 단순한 기업의 책무로서 사회공헌에 임하는...
1268,1269,는 첫째 전공 과정을 수료함으로써 생산에 있어 발생하는 대부분의 문제에 대해 효율적...


In [3]:
text = data['content']
text

0       봉사활동을 참여하게 된 후로부터 저는 저의 도움으로 다른 사람들이 행복해지는 모습을...
1       노력파 현재진행형 인재저는 컴퓨터 전기전자과를 전공하여 전기 전자와 관련된 여러 과...
2       설계의 꿈을 갖고 기초공사를 마치다제가 설계 직무에 적합한 이유로 3가지가 있습니다...
3       감동을 주는 엔지니어응용공학설계 프로젝트에서 차량 프로젝트로 설계와 구조해석을 하면...
4       기업 솔루션 종결자 후지제록스에서 꿈을 펼치다!후지제록스는 많은 기업이 직면해 있는...
                              ...                        
1265    수동적이고 기계적인 태도로 주어진 기본 업무만 처리하는 사람은 능동적으로 업무를 완...
1266    우리 사회가 고령화 사회로 진입하게 되면서 사람들은 단순히 오래 사는 것보다는 건강...
1267    귀사의 사회공헌활동에는 역동성이 있습니다. 단순한 기업의 책무로서 사회공헌에 임하는...
1268    는 첫째 전공 과정을 수료함으로써 생산에 있어 발생하는 대부분의 문제에 대해 효율적...
1269    제품 홍보가 곧 기업 홍보다제품 홍보가 곧 기업 홍보라는 회사의 모토처럼 학창시절부...
Name: content, Length: 1270, dtype: object

In [4]:
fulltext= ""
for i in range(50):
    fulltext += str(text[i])+'\n' #일단 조금만

In [5]:
t = Tokenizer()
t.fit_on_texts([fulltext])
vocab_size= len(t.word_index)+1
print('단어 집합의 크기 : %d'%vocab_size)

단어 집합의 크기 : 4080


In [6]:
print(t.word_index)

{'수': 1, '있는': 2, '통해': 3, '대한': 4, '되었습니다': 5, '있습니다': 6, '저는': 7, '있었습니다': 8, '위해': 9, '그': 10, '많은': 11, '새로운': 12, '싶습니다': 13, '더': 14, '제가': 15, '그리고': 16, '가치를': 17, '이를': 18, '것입니다': 19, '대해': 20, '또한': 21, '했습니다': 22, '생각합니다': 23, '제': 24, '때문입니다': 25, '때': 26, '하고': 27, '등': 28, '한': 29, '지원하게': 30, '될': 31, '일을': 32, '저의': 33, '것을': 34, '및': 35, '되겠습니다': 36, '알게': 37, '다양한': 38, '가장': 39, '함께': 40, '직접': 41, '하는': 42, '때문에': 43, '할': 44, '관심을': 45, '큰': 46, '바탕으로': 47, '기업의': 48, '지식을': 49, '것이': 50, '등을': 51, '여러': 52, '등의': 53, '업무를': 54, '서비스를': 55, '이러한': 56, '있도록': 57, '그래서': 58, '주고': 59, '후': 60, '아니라': 61, '않는': 62, '지원했습니다': 63, '항상': 64, '꿈을': 65, '되었고': 66, '하면서': 67, '싶어': 68, '일': 69, '모습을': 70, '컴퓨터': 71, '하지만': 72, '역량을': 73, '경험을': 74, '관련': 75, '꾸준히': 76, '고객의': 77, '동안': 78, '능력을': 79, '않았습니다': 80, '회계': 81, '제품을': 82, '않고': 83, '있다는': 84, '갖고': 85, '설계': 86, '도움이': 87, '배웠습니다': 88, '입사': 89, '고객': 90, '있게': 91, '학과': 92, '학습을': 93, '전': 94, '위한': 95, '대학': 

In [7]:
sequences = list()
for line in fulltext.split('\n'): # Wn을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 7366


In [8]:
max_len=max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 240


In [9]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [10]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.

In [11]:
y = to_categorical(y, num_classes=vocab_size)
#원핫인코딩

In [12]:
# RNN 모델
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [13]:
model = Sequential()
model.add(Embedding(vocab_size, 20, input_length=max_len-1))
# model.add(SimpleRNN(32))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 239, 20)           81600     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               76288     
_________________________________________________________________
dense (Dense)                (None, 4080)              526320    
Total params: 684,208
Trainable params: 684,208
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
231/231 [==============================] - 22s 97ms/step - loss: 8.2630 - accuracy: 0.0136
Epoch 2/200
231/231 [==============================] - 24s 104ms/step - loss: 7.9435 - accuracy: 0.0148
Epoch 3/200
231/231 [==============================] - 24s 104ms/step - loss: 7.7069 - accuracy: 0.0159
Epoch 4/200
231/231 [==============================] - 24s 104ms/step - loss: 7.4568 - accuracy: 0.0167
Epoch 5/200
231/231 [==============================] - 24s 105ms/step - loss: 7.1548 - accuracy: 0.0185
Epoch 6/200
231/231 [==============================] - 24s 104ms/step - loss: 6.7241 - accuracy: 0.0262
Epoch 7/200
231/231 [==============================] - 24s 104ms/step - loss: 6.2474 - accuracy: 0.0375
Epoch 8/200
231/231 [==============================] - 24s 104ms/step - loss: 5.7501 - accuracy: 0.0504
Epoch 9/200
231/231 [==============================] - 24s 104ms/step - loss: 5.3247 - accuracy: 0.0732
Epoch 10/200
231/231 [==============================] - 24s 104ms

In [16]:
#출력함수
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [24]:
sentence_generation(model, t, '첫째', 10)

'첫째 인턴 주는 고민을 했습니다 많은 자세히 집중이라는 그룹 자전거를 타고'